In [0]:
import pandas as pd
import numpy as np
import os
import tensorflow.keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from keras.optimizers import Adam
from google.colab import files
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input

In [35]:
import os
os.environ['KAGGLE_USERNAME'] = "******"
os.environ['KAGGLE_KEY'] = "****" # key from the json file
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia # api copied from kaggle

 99% 1.15G/1.15G [00:07<00:00, 112MB/s]
100% 1.15G/1.15G [00:07<00:00, 155MB/s]


In [36]:
!apt install pv
!unzip -o /content/chest-xray-pneumonia.zip  | pv -l >/dev/null
os.remove('chest-xray-pneumonia.zip')
!unzip -o /content/chest_xray.zip  | pv -l >/dev/null
os.remove('chest_xray.zip')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pv is already the newest version (1.6.6-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
2.00  0:00:09 [ 207m/s] [  <=>                                                 ]
11.7k 0:00:14 [ 785 /s] [               <=>                                    ]


In [0]:
base_model=InceptionV3(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

In [0]:
model=Model(inputs=base_model.input,outputs=preds)

for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [39]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/content/chest_xray/train/',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=32,
                                                  class_mode='categorical', shuffle=True)


model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5)

Found 5216 images belonging to 2 classes.
Epoch 1/5
163/163 [==============================] - 111s 681ms/step - loss: 0.1506 - acc: 0.9503
Epoch 2/5
163/163 [==============================] - 90s 555ms/step - loss: 0.0539 - acc: 0.9810
Epoch 3/5
163/163 [==============================] - 90s 553ms/step - loss: 0.0468 - acc: 0.9820
Epoch 4/5
163/163 [==============================] - 90s 553ms/step - loss: 0.0388 - acc: 0.9866
Epoch 5/5
163/163 [==============================] - 90s 552ms/step - loss: 0.0829 - acc: 0.9770


In [29]:
model.save('chest-xray-pneumonia.h5')    
!zip -r model.zip 'chest-xray-pneumonia.h5'

updating: chest-xray-pneumonia.h5 (deflated 10%)


In [40]:
from tensorflow.keras.models import load_model
new_model = load_model("/content/chest-xray-pneumonia.h5")

def get_rez(pic):
  img = image.load_img(pic, target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  p_good,p_ill = np.around(new_model.predict(x), decimals=2)[0]
  return{'p_good':p_good,'p_ill':p_ill}




Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [41]:
ill_path = "/content/chest_xray/train/PNEUMONIA/" 
good_path = "/content/chest_xray/train/NORMAL/" 

ill_pic = ill_path + os.listdir(ill_path)[0]
good_pic = good_path + os.listdir(good_path)[0]

print(get_rez(ill_pic))
print(get_rez(good_pic))

{'p_good': 0.0, 'p_ill': 1.0}
{'p_good': 0.66, 'p_ill': 0.34}
